In [1]:
import re
import pickle

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm

In [2]:
tf.__version__

'2.3.0'

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  print("Invalid device or cannot modify virtual devices once initialized.")
  pass

In [4]:
def preprocess(string):
    string = string.lower()
    return re.sub(r"[\W_]+", " ", string)

In [5]:
train_df = pd.read_json('data/train.jsonl', lines = True)
train_df["img"] = "data/" + train_df["img"]
train_df["text"] = train_df["text"].apply(preprocess)

In [6]:
val_df = pd.read_json('data/dev.jsonl', lines = True)
val_df["img"] = "data/" + val_df["img"]
val_df["text"] = val_df["text"].apply(preprocess)

In [7]:
img_width = 128
img_height = 128

In [8]:
embeds = pickle.load(open("data/embeds.pickle", "rb"))
val_embeds = pickle.load(open("data/val-embeds.pickle", "rb"))

In [9]:
def build_model():
    input_img = tf.keras.layers.Input(
        shape = (img_width, img_height, 3), name = "image"
    )
    
    embeds = tf.keras.layers.Input(name = "embeds", shape = (768,))

    extractor = tf.keras.applications.EfficientNetB0(include_top = False, \
                                                 input_tensor = input_img, weights = "imagenet")
    
    extractor.trainable = False
    
    for layer in extractor.layers:
        if layer.name.startswith("block7"):
            layer.trainable = True

    x = tf.keras.layers.GlobalMaxPooling2D()(extractor.output)
    
    x = tf.keras.layers.concatenate([x, embeds])
    
    x = tf.keras.layers.BatchNormalization()(x)
    
    x = tf.keras.layers.Dense(128, activation = "relu")(x)
    
    x = tf.keras.layers.Dropout(0.2)(x)
    
    out = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

    model = tf.keras.models.Model([input_img, embeds], out)

    model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
    return model

In [10]:
model = build_model()
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image (InputLayer)              [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 128, 128, 3)  0           image[0][0]                      
__________________________________________________________________________________________________
normalization (Normalization)   (None, 128, 128, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 129, 129, 3)  0           normalization[0][0]              
_______________________________________________________________________________________

In [11]:
def encode_single_sample(img_path, label, embeds):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_png(img, channels = 3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, [img_height, img_width])
    return {"image": img, "label": label, "embeds": embeds}

In [12]:
class HatefulMemes(tf.keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, input_img_paths, label, embeds):
        self.batch_size = batch_size
        self.input_img_paths = input_img_paths
        self.embeds = embeds
        self.label = label

    def __len__(self):
        return len(self.input_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_label = self.label[i : i + self.batch_size]
        batch_embeds = self.embeds[i : i + self.batch_size]
        x = np.zeros((self.batch_size,) + (img_height, img_width) + (3,), dtype = "float32")
        y = np.zeros((self.batch_size,) + (768,), dtype = "float32")
        z = np.zeros((self.batch_size,), dtype = "uint8")
        
        for i,j,k,l in zip(range(self.batch_size), batch_input_img_paths, batch_label, batch_embeds):
            sample = encode_single_sample(j,k,l)
            x[i] = sample["image"].numpy().tolist()
            y[i] = sample["embeds"]
            z[i] = sample["label"]
        
        return [x,y], z

In [13]:
train_gen = HatefulMemes(8, train_df["img"].values.tolist()[:7500], train_df["label"].values.tolist()[:7500], embeds[:7500])
val_gen = HatefulMemes(8, train_df["img"].values.tolist()[7500:], train_df["label"].values.tolist()[7500:], embeds[7500:])

In [ ]:
epochs = 100
early_stopping_patience = 5

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = "val_accuracy", patience = early_stopping_patience, restore_best_weights = True
)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'hateful.h5', monitor = 'val_accuracy', verbose = 1, save_best_only = True, mode = 'max'
)

# Train the model
history = model.fit(
    train_gen,
    validation_data = val_gen, 
    epochs = epochs,
    callbacks = [early_stopping, checkpoint]
)

Epoch 1/100
 18/937 [..............................] - ETA: 4:12 - loss: 1.5608 - accuracy: 0.4931